# Route Optimization Engine

## Problem Description
1) Diberikan file Xlsx berisi 317 data dengan kolom ID, address, coordinate, open time, close time, visit tag & time, Jadwal kunjung / bulan
2) Lakukan routing seminggu sekali dalam sebulan dengan ketentuan:
- Satu customer hanya sekali kunjungan, setelah itu drop customer itu
- Apabila ada keterangan 2 kunjungan/month, maka ada jeda seminggu
- Koordinat Depot: -8.2159423, 114.3694888
3) Cari tau customer mana yang mau routing minggu 1, minggu 2, ... , minggu 4
4) Ada 82 customer yang di-routing tiap minggu
5) Per vehicle ada 20 titik yang dikunjungi
6) Maksimum waktu kunjung per vehicle adalah 5 jam 30 menit 
7) Tambah kolom baru berupa grouping berdasarkan approach jadi gaperlu drop satu per satu

## Approach
- Random Approach
- Minimum latitude approach
- Minimum longitude approach

In [1]:
pip install pandas openpyxl

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


### Data Exploration

In [2]:
import pandas as pd
df = pd.read_excel('data/input/Field Sales_template (3) BWI001.xlsx')

In [3]:
df

,Cutomer Name,Customer Address,Customer Coordinate,Open Time,Close Time,Visit Tag,Visit time,Jadwal Kunjungan / month
0,CWRK009-00-0155,JL DR SUTOMO 27 Banyuwangi Kab. Banyuwangi Jaw...,"-8.21448,114.37239",09:00:00,14:30:00,NaN,15,2
1,CWRK009-00-0033,DALAM PASAR NO 61 Banyuwangi Kab. Banyuwangi J...,"-8.21091,114.3742",09:00:00,14:30:00,NaN,15,1
2,CWRK009-00-0034,JL A YANI NO 10 Banyuwangi Kab. Banyuwangi Jaw...,"-8.2163,114.37007",09:00:00,14:30:00,NaN,15,1
3,CWRK009-00-0421,JL. A YANI PENGAJUARAN BLOCK NO 21-23 Banyuwan...,"-8.217970730504070,114.36980812364700",09:00:00,14:30:00,NaN,12,1
4,CWRK009-00-0010,JL SAYUWIWIT 38 Banyuwangi Kab. Banyuwangi Jaw...,"-8.20831,114.37638",09:00:00,14:30:00,NaN,15,1
...,...,...,...,...,...,...,...,...
312,CWRK009-00-0700,PEREMPATAN PASAR SRATEN Cluring Kab. Banyuwang...,"-8.417143,114.246739",09:00:00,14:30:00,NaN,12,1
313,WRK009-21-0000053,JL RAYA JEMBER GLENMORE Glenmore Kab. Banyuwan...,"-8.31693,114.05773",09:00:00,14:30:00,NaN,12,1
314,WRK009-23-0000034,DS KARANGREJO NO 3 GLENMORE Glenmore Kab. Bany...,"-8.31527,114.05496",09:00:00,14:30:00,NaN,12,1
315,WRK009-21-0000032,KEMIREN BANYUWANGI Licin Kab. Banyuwangi Jawa ...,"-8.20467,114.318933",09:00:00,14:30:00,NaN,12,1


In [4]:
# Misahin latitude dan longitude untuk bisa masuk ke algoritma
df[['latitude', 'longitude']] = df['Customer Coordinate'].str.split(',', expand=True)

In [5]:
data_baru = 'Field Saless.xlsx'
df.to_excel(data_baru, index=False)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317 entries, 0 to 316
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Cutomer Name              317 non-null    object 
 1   Customer Address          317 non-null    object 
 2   Customer Coordinate       317 non-null    object 
 3   Open Time                 317 non-null    object 
 4   Close Time                317 non-null    object 
 5   Visit Tag                 0 non-null      float64
 6   Visit time                317 non-null    int64  
 7   Jadwal Kunjungan / month  317 non-null    int64  
 8   latitude                  317 non-null    object 
 9   longitude                 317 non-null    object 
dtypes: float64(1), int64(2), object(7)
memory usage: 24.9+ KB


In [7]:
df.min()

Cutomer Name                                          CWRK009-00-0009
Customer Address            BAGOREJO Srono Kab. Banyuwangi Jawa Timur
Customer Coordinate                              -7.930725,114.386792
Open Time                                                    09:00:00
Close Time                                                   14:30:00
Visit Tag                                                         NaN
Visit time                                                         12
Jadwal Kunjungan / month                                            1
latitude                                                    -7.930725
longitude                                          114.09942531089538
dtype: object

In [8]:
df.max()

Cutomer Name                                               WRK009-23-0000154
Customer Address            WONGSOREJO Banyuwangi Kab. Banyuwangi Jawa Timur
Customer Coordinate                                      -8.594555,114.22332
Open Time                                                           09:00:00
Close Time                                                          14:30:00
Visit Tag                                                                NaN
Visit time                                                                15
Jadwal Kunjungan / month                                                   2
latitude                                                           -8.594555
longitude                                                          114.40437
dtype: object

In [9]:
value_counts = df['Jadwal Kunjungan / month'].value_counts()
value_counts

Jadwal Kunjungan / month
1    306
2     11
Name: count, dtype: int64

In [10]:
jadwal_kunjung2 = df[df['Jadwal Kunjungan / month'] == 2]
grouping_jadwal = jadwal_kunjung2['Cutomer Name'].unique()

print("Customer 2 kali kunjungan / bulan = 2:")
for customer in grouping_jadwal:
    print(customer)

Customer 2 kali kunjungan / bulan = 2:
CWRK009-00-0155
CWRK009-00-0082
CWRK009-00-0022
CWRK009-00-0214
WRK009-19-0000009
CWRK009-00-0111
CWRK009-00-0186
CWRK009-00-0356
CWRK009-00-0099
CWRK009-00-0514
WRK009-19-0000003


In [11]:
df

,Cutomer Name,Customer Address,Customer Coordinate,Open Time,Close Time,Visit Tag,Visit time,Jadwal Kunjungan / month,latitude,longitude
0,CWRK009-00-0155,JL DR SUTOMO 27 Banyuwangi Kab. Banyuwangi Jaw...,"-8.21448,114.37239",09:00:00,14:30:00,NaN,15,2,-8.21448,114.37239
1,CWRK009-00-0033,DALAM PASAR NO 61 Banyuwangi Kab. Banyuwangi J...,"-8.21091,114.3742",09:00:00,14:30:00,NaN,15,1,-8.21091,114.3742
2,CWRK009-00-0034,JL A YANI NO 10 Banyuwangi Kab. Banyuwangi Jaw...,"-8.2163,114.37007",09:00:00,14:30:00,NaN,15,1,-8.2163,114.37007
3,CWRK009-00-0421,JL. A YANI PENGAJUARAN BLOCK NO 21-23 Banyuwan...,"-8.217970730504070,114.36980812364700",09:00:00,14:30:00,NaN,12,1,-8.217970730504070,114.36980812364700
4,CWRK009-00-0010,JL SAYUWIWIT 38 Banyuwangi Kab. Banyuwangi Jaw...,"-8.20831,114.37638",09:00:00,14:30:00,NaN,15,1,-8.20831,114.37638
...,...,...,...,...,...,...,...,...,...,...
312,CWRK009-00-0700,PEREMPATAN PASAR SRATEN Cluring Kab. Banyuwang...,"-8.417143,114.246739",09:00:00,14:30:00,NaN,12,1,-8.417143,114.246739
313,WRK009-21-0000053,JL RAYA JEMBER GLENMORE Glenmore Kab. Banyuwan...,"-8.31693,114.05773",09:00:00,14:30:00,NaN,12,1,-8.31693,114.05773
314,WRK009-23-0000034,DS KARANGREJO NO 3 GLENMORE Glenmore Kab. Bany...,"-8.31527,114.05496",09:00:00,14:30:00,NaN,12,1,-8.31527,114.05496
315,WRK009-21-0000032,KEMIREN BANYUWANGI Licin Kab. Banyuwangi Jawa ...,"-8.20467,114.318933",09:00:00,14:30:00,NaN,12,1,-8.20467,114.318933


In [12]:
import folium
def create_map_and_plot_routes(depot, rute_stops_df, center_location):
    m = folium.Map(location=center_location, zoom_start=10)
    folium.Marker(location=depot, icon=folium.Icon(color='red', icon='home', prefix='fa'), tooltip="Depot").add_to(m)
    for index, rute_stop in rute_stops_df.iterrows():
        stop_coords = (rute_stop['latitude'], rute_stop['longitude'])
        folium.Marker(location=stop_coords, icon=folium.Icon(color='green', icon='circle', prefix='fa'), tooltip="Pemberhentian").add_to(m)
    return m

depot_location = (-8.2159423, 114.3694888)
center_location = depot_location

# Buat plot peta OpenStreetMap
customer_map = create_map_and_plot_routes(depot_location, df, center_location)
customer_map

# 1) Random approach
- Grouping per week
- Buat DataFrame baru untuk random approach

In [5]:
df_rand = df.copy()
df_rand['Month Rand'] = df_rand['Cutomer Name']

In [14]:
df_rand

,Cutomer Name,Customer Address,Customer Coordinate,Open Time,Close Time,Visit Tag,Visit time,Jadwal Kunjungan / month,latitude,longitude,Month Rand
0,CWRK009-00-0155,JL DR SUTOMO 27 Banyuwangi Kab. Banyuwangi Jaw...,"-8.21448,114.37239",09:00:00,14:30:00,NaN,15,2,-8.21448,114.37239,CWRK009-00-0155
1,CWRK009-00-0033,DALAM PASAR NO 61 Banyuwangi Kab. Banyuwangi J...,"-8.21091,114.3742",09:00:00,14:30:00,NaN,15,1,-8.21091,114.3742,CWRK009-00-0033
2,CWRK009-00-0034,JL A YANI NO 10 Banyuwangi Kab. Banyuwangi Jaw...,"-8.2163,114.37007",09:00:00,14:30:00,NaN,15,1,-8.2163,114.37007,CWRK009-00-0034
3,CWRK009-00-0421,JL. A YANI PENGAJUARAN BLOCK NO 21-23 Banyuwan...,"-8.217970730504070,114.36980812364700",09:00:00,14:30:00,NaN,12,1,-8.217970730504070,114.36980812364700,CWRK009-00-0421
4,CWRK009-00-0010,JL SAYUWIWIT 38 Banyuwangi Kab. Banyuwangi Jaw...,"-8.20831,114.37638",09:00:00,14:30:00,NaN,15,1,-8.20831,114.37638,CWRK009-00-0010
...,...,...,...,...,...,...,...,...,...,...,...
312,CWRK009-00-0700,PEREMPATAN PASAR SRATEN Cluring Kab. Banyuwang...,"-8.417143,114.246739",09:00:00,14:30:00,NaN,12,1,-8.417143,114.246739,CWRK009-00-0700
313,WRK009-21-0000053,JL RAYA JEMBER GLENMORE Glenmore Kab. Banyuwan...,"-8.31693,114.05773",09:00:00,14:30:00,NaN,12,1,-8.31693,114.05773,WRK009-21-0000053
314,WRK009-23-0000034,DS KARANGREJO NO 3 GLENMORE Glenmore Kab. Bany...,"-8.31527,114.05496",09:00:00,14:30:00,NaN,12,1,-8.31527,114.05496,WRK009-23-0000034
315,WRK009-21-0000032,KEMIREN BANYUWANGI Licin Kab. Banyuwangi Jawa ...,"-8.20467,114.318933",09:00:00,14:30:00,NaN,12,1,-8.20467,114.318933,WRK009-21-0000032


In [6]:
import numpy as np

In [7]:
customer_name_dual = [
    'CWRK009-00-0155', 'CWRK009-00-0082', 'CWRK009-00-0022', 'CWRK009-00-0214', 
    'WRK009-19-0000009', 'CWRK009-00-0111', 'CWRK009-00-0186', 'CWRK009-00-0356', 
    'CWRK009-00-0099', 'CWRK009-00-0514', 'WRK009-19-0000003'
]

# Membuat dataframe baru dengan menambahkan kolom 'Month Rand'
df_rand = df.copy()
df_rand['Month Rand'] = df_rand['Cutomer Name']

# Mendapatkan data yang memiliki nilai 2 pada 'Jadwal Kunjungan / month'
df_dual = df_rand[df_rand['Cutomer Name'].isin(customer_name_dual)].copy()
df_dual_1 = df_dual.copy()
df_dual_2 = df_dual.copy()

# Mengatur 'Month Rand' untuk df_dual_1 dan df_dual_2 dengan jadwal yang tidak berurutan
schedule = ['W1', 'W2', 'W3', 'W4']
df_dual_1['Month Rand'] = np.tile(schedule[:2], len(df_dual_1) // 2 + 1)[:len(df_dual_1)]
df_dual_2['Month Rand'] = np.tile(schedule[2:], len(df_dual_2) // 2 + 1)[:len(df_dual_2)]

# Menggabungkan df_dual_1 dan df_dual_2 ke dalam df_rand
df_rand = pd.concat([df_rand, df_dual_2])

# Menghapus baris asli dari df_rand yang memiliki 'Cutomer Name' pada customer_name_dual
df_rand = df_rand[~df_rand.duplicated(subset=['Cutomer Name', 'Month Rand'], keep=False)]

# Menyusun ulang dan membagi data lainnya ke dalam W1, W2, W3, W4
remaining_customers = df_rand[~df_rand['Cutomer Name'].isin(customer_name_dual)]
remaining_customers = remaining_customers.sample(frac=1).reset_index(drop=True)
remaining_customers['Month Rand'] = np.tile(schedule, len(remaining_customers) // 4 + 1)[:len(remaining_customers)]

# Menggabungkan kembali semuanya
df_rand = pd.concat([df_dual_1, df_dual_2, remaining_customers]).reset_index(drop=True)

In [8]:
df_rand

,Cutomer Name,Customer Address,Customer Coordinate,Open Time,Close Time,Visit Tag,Visit time,Jadwal Kunjungan / month,latitude,longitude,Month Rand
0,CWRK009-00-0155,JL DR SUTOMO 27 Banyuwangi Kab. Banyuwangi Jaw...,"-8.21448,114.37239",09:00:00,14:30:00,NaN,15,2,-8.21448,114.37239,W1
1,CWRK009-00-0082,JL IKAN TOMBRO 28 Banyuwangi Kab. Banyuwangi J...,"-8.21923,114.37842",09:00:00,14:30:00,NaN,15,2,-8.21923,114.37842,W2
2,CWRK009-00-0022,JL MH THAMRIN 26 Banyuwangi Kab. Banyuwangi Ja...,"-8.21953,114.37394",09:00:00,14:30:00,NaN,12,2,-8.21953,114.37394,W1
3,CWRK009-00-0214,DPN POM BENSIN JAJAG RT7 RW 3 Gambiran Kab. Ba...,"-8.449566,114.182938",09:00:00,14:30:00,NaN,15,2,-8.449566,114.182938,W2
4,WRK009-19-0000009,JL RAYA SETINGGIL NO 35 MUNCAR Muncar Kab. Ban...,"-8.431568,114.33768",09:00:00,14:30:00,NaN,15,2,-8.431568,114.33768,W1
...,...,...,...,...,...,...,...,...,...,...,...
323,WRK009-23-0000066,JL SUKOREJO SILIRAGUNG Siliragung Kab. Banyuwa...,"-8.565759,114.109164",09:00:00,14:30:00,NaN,12,1,-8.565759,114.109164,W2
324,CWRK009-00-0822,JL RAYA TAMPO KRADENAN Purwoharjo Kab. Banyuwa...,"-8.463496,114.232883",09:00:00,14:30:00,NaN,12,1,-8.463496,114.232883,W3
325,WRK009-22-0000045,JL BASUKI RAHMAT BANYUWANGI Banyuwangi Kab. Ba...,"-8.194094,114.376183",09:00:00,14:30:00,NaN,12,1,-8.194094,114.376183,W4
326,WRK009-21-0000055,JL JOYOBOYO NO 32 KALIPURO BANYUWANGI Kalipuro...,"-8.17742421675588,114.36407148930300",09:00:00,14:30:00,NaN,12,1,-8.17742421675588,114.36407148930300,W1


### Week 1

In [10]:
df_w1 = df_rand[df_rand['Month Rand'] == 'W1']
df_w1.loc[:, 'Month Rand'] = ['W1' + str(i + 1) for i in range(len(df_w1))]
df_w1.head()

,Cutomer Name,Customer Address,Customer Coordinate,Open Time,Close Time,Visit Tag,Visit time,Jadwal Kunjungan / month,latitude,longitude,Month Rand
0,CWRK009-00-0155,JL DR SUTOMO 27 Banyuwangi Kab. Banyuwangi Jaw...,"-8.21448,114.37239",09:00:00,14:30:00,NaN,15,2,-8.21448,114.37239,W11
2,CWRK009-00-0022,JL MH THAMRIN 26 Banyuwangi Kab. Banyuwangi Ja...,"-8.21953,114.37394",09:00:00,14:30:00,NaN,12,2,-8.21953,114.37394,W12
4,WRK009-19-0000009,JL RAYA SETINGGIL NO 35 MUNCAR Muncar Kab. Ban...,"-8.431568,114.33768",09:00:00,14:30:00,NaN,15,2,-8.431568,114.33768,W13
6,CWRK009-00-0186,DPN PUSKESMAS PESANGGRAHAN 63 Pesanggaran Kab....,"-8.564528,114.09949",09:00:00,14:30:00,NaN,15,2,-8.564528,114.09949,W14
8,CWRK009-00-0099,JL RAYA SUMBER WADUNG Kabat Kab. Banyuwangi...,"-8.301742,114.105952",09:00:00,14:30:00,NaN,15,2,-8.301742,114.105952,W15
...,...,...,...,...,...,...,...,...,...,...,...
310,WRK009-23-0000139,JL AYANI NO 267 PAKIS KEC SONGGON Songgon Kab....,"-8.236884,114.206584",09:00:00,14:30:00,NaN,12,1,-8.236884,114.206584,W179
314,CWRK009-00-0506,PASAR PESANGGRAHAN Pesanggaran Kab. Banyuwangi...,"-8.564447243230823, 114.09942531089538",09:00:00,14:30:00,NaN,12,1,-8.564447243230823,114.09942531089538,W180
318,CWRK009-00-0906,JL HASSANUDIN GENTENG Genteng Kab. Banyuwangi ...,"-8.364849,114.158678",09:00:00,14:30:00,NaN,15,1,-8.364849,114.158678,W181
322,WRK009-22-0000029,JL ARUJI KERTAWINATA DSN JURUH Singojuruh Kab....,"-8.309427,114.241411",09:00:00,14:30:00,NaN,12,1,-8.309427,114.241411,W182


In [19]:
df_w1.to_excel('Field Sales W1 Random.xlsx', index=False)

In [20]:
grouping_w1 = df_w1.groupby(["Cutomer Name"])[["latitude", "longitude"]].min().reset_index()
grouping_w1

,Cutomer Name,latitude,longitude
0,CWRK009-00-0014,-8.24311,114.35274
1,CWRK009-00-0018,-8.402006,114.263651
2,CWRK009-00-0022,-8.21953,114.37394
3,CWRK009-00-0023,-8.00268,114.40437
4,CWRK009-00-0031,-8.364594,114.157678
...,...,...,...
78,WRK009-23-0000129,-8.212062727193500,114.37706858748800
79,WRK009-23-0000139,-8.236884,114.206584
80,WRK009-23-0000141,-8.251822,114.358719
81,WRK009-23-0000144,-8.534945936522600,114.16727378096900


In [21]:
grouping_w1.to_excel('Field Sales W1 Random Result.xlsx', index=False)

In [22]:
pip install tenacity

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [23]:
import contextily as ctx
import folium
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
import geopy.distance
from folium import plugins
from operator import itemgetter
from scipy.spatial import distance
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2
import pandas as pd
from tenacity import retry, stop_after_attempt, wait_exponential

def load_data(file_path):
    return pd.read_excel(file_path)

def group_and_calculate_median(data):
    district_tempat = data.groupby('Cutomer Name')[['latitude', 'longitude']].median().reset_index()
    district_tempat.columns = ['Cutomer Name', 'latitude', 'longitude']
    return district_tempat

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def get_highway_graph(center_location, dist):
    G = ox.graph_from_point(center_location, dist=dist, network_type='drive')
    return ox.utils_graph.get_largest_component(G, strongly=True)

def get_depot_and_nearest_nodes(G, center_location, district_tempat):
    depot = ox.distance.nearest_nodes(G, center_location[1], center_location[0])
    rute_stops = [(row['latitude'], row['longitude']) for _, row in district_tempat.iterrows()]
    node_stop = [ox.distance.nearest_nodes(G, stop[1], stop[0]) for stop in rute_stops]
    return depot, node_stop

def add_bus_stops_to_graph(G, district_tempat):
    for _, rute_stop in district_tempat.iterrows():
        nearest_node = ox.distance.nearest_nodes(G, rute_stop['longitude'], rute_stop['latitude'])
        dist = geopy.distance.distance((G.nodes[nearest_node]['y'], G.nodes[nearest_node]['x']), (rute_stop['latitude'], rute_stop['longitude']))
        G.add_node(rute_stop['Cutomer Name'], x=rute_stop['longitude'], y=rute_stop['latitude'])
        G.add_edge(nearest_node, rute_stop['Cutomer Name'], weight=dist.m)
        G.add_edge(rute_stop['Cutomer Name'], nearest_node, weight=dist.m)
    return G

def create_routing_model(nodes, NUM_VEHICLES, depot):
    manager = pywrapcp.RoutingIndexManager(len(nodes), NUM_VEHICLES, nodes.index(depot))
    routing = pywrapcp.RoutingModel(manager)
    return routing, manager

def distance_callback(from_node_index, to_node_index, nodes, G, manager):
    from_node = nodes[manager.IndexToNode(from_node_index)]
    to_node = nodes[manager.IndexToNode(to_node_index)]
    return nx.shortest_path_length(G, from_node, to_node)

def add_distance_constraint(routing, transit_callback_index, NUM_VEHICLES):
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,
        3000,
        True,
        dimension_name
    )
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

def solve_routing_problem(routing, search_parameters):
    return routing.SolveWithParameters(search_parameters)

def print_solution(routing, solution, NUM_VEHICLES, manager, nodes, district_tempat):
    node_to_place_id = {0: 'depot'}
    for i, node in enumerate(nodes[1:], start=1):
        node_to_place_id[node] = district_tempat.iloc[i-1]['Cutomer Name']
    
    total_distance = 0
    for vehicle_id in range(NUM_VEHICLES):
        index = routing.Start(vehicle_id)
        route_distance = 0
        route = ['depot']  # Start with 'depot'
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            if node_index != 0:  # Add all nodes except the depot
                route.append(node_to_place_id[nodes[node_index]])
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        route.append('depot')  # End with 'depot'
        print(f"Route for vehicle {vehicle_id}:\n{' -> '.join(str(node) for node in route)}")
        print(f"Distance of route: {route_distance/10} km\n")
        total_distance += route_distance
    print(f"Total distance of all routes: {total_distance/10} km")


def create_map_and_plot_routes(G, depot, rute_stops_df, nodes, solution, routing, manager, NUM_VEHICLES, center_location):
    m = folium.Map(location=center_location, zoom_start=16)
    
    # Add depot marker
    depot_coords = (G.nodes[depot]['y'], G.nodes[depot]['x'])
    folium.Marker(location=depot_coords, icon=folium.Icon(color='red', icon='home', prefix='fa'), tooltip=f"Depot {depot_coords}").add_to(m)
    
    # Add stop markers
    for index, rute_stop in rute_stops_df.iterrows():
        stop_coords = (rute_stop['latitude'], rute_stop['longitude'])
        folium.Marker(location=stop_coords, icon=folium.Icon(color='green', icon='circle', prefix='fa'), tooltip=f"Pemberhentian {stop_coords}").add_to(m)
    
    # Colors for different vehicle routes
    colors = ['blue', 'orange', 'yellow', 'green']
    
    for vehicle_id in range(NUM_VEHICLES):
        index = routing.Start(vehicle_id)
        route = []
        
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route.append(nodes[node_index])
            index = solution.Value(routing.NextVar(index))
        
        route.append(nodes[manager.IndexToNode(index)])  # Add depot at the end
        color = colors[vehicle_id % len(colors)]
        
        # Plot the route
        for i in range(len(route) - 1):
            path = nx.shortest_path(G, route[i], route[i + 1], weight='length')
            path_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in path]
            
            folium.PolyLine(locations=path_coords, color=color, weight=5, tooltip=f"Vehicle {vehicle_id}").add_to(m)
            ant_path = plugins.AntPath(
                locations=path_coords,
                color=color,
                dash_array=[10, 50],
                delay=500,
                weight=5,
            )
            m.add_child(ant_path)
    
    return m

def optimize_delivery_routes(file_path, center_location, dist, NUM_VEHICLES):
    # Load data
    data = load_data(file_path)
    
    # Group and calculate median
    district_tempat = group_and_calculate_median(data)
    
    # Get the highway graph
    try:
        G = get_highway_graph(center_location, dist)
    except Exception as e:
        print(f"Failed to download graph data: {e}")
        return None
    
    # Get depot and nearest nodes
    depot, node_stop = get_depot_and_nearest_nodes(G, center_location, district_tempat)
    
    # Add bus stops to the highway graph
    G = add_bus_stops_to_graph(G, district_tempat)
    
    # Create routing model
    nodes = [depot] + node_stop
    routing, manager = create_routing_model(nodes, NUM_VEHICLES, depot)
    
    # Define distance callback
    transit_callback_index = routing.RegisterTransitCallback(lambda from_node, to_node: distance_callback(from_node, to_node, nodes, G, manager))
    
    # Add Distance constraint
    add_distance_constraint(routing, transit_callback_index, NUM_VEHICLES)
    
    # Set path-cheapest-arc search strategy
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    
    # Solve the problem
    solution = solve_routing_problem(routing, search_parameters)
    
    # Print the solution
    print_solution(routing, solution, NUM_VEHICLES, manager, nodes, district_tempat)
    
    # Create map and plot routes
    m = create_map_and_plot_routes(G, depot, district_tempat, nodes, solution, routing, manager, NUM_VEHICLES, center_location)
    
    # Display map
    return m

file_path = "Field Sales W1 Random.xlsx"
center_location = (-8.2159423, 114.3694888)
dist = 30000
NUM_VEHICLES = 4

In [24]:
optimized_delivery_map = optimize_delivery_routes(file_path, center_location, dist, NUM_VEHICLES)

Route for vehicle 0:
depot -> CWRK009-00-0155 -> CWRK009-00-0022 -> WRK009-23-0000141 -> CWRK009-00-0360 -> WRK009-19-0000009 -> WRK009-21-0000082 -> CWRK009-00-0217 -> CWRK009-00-0238 -> CWRK009-00-0278 -> CWRK009-00-0402 -> WRK009-20-0000063 -> WRK009-20-0000063 -> WRK009-20-0000063 -> WRK009-20-0000063 -> WRK009-20-0000063 -> WRK009-20-0000063 -> WRK009-23-0000104 -> WRK009-23-0000104 -> CWRK009-00-0930 -> CWRK009-00-0867 -> CWRK009-00-0476 -> CWRK009-00-0300 -> CWRK009-00-0326 -> CWRK009-00-0225 -> CWRK009-00-0105 -> WRK009-21-0000096 -> WRK009-21-0000058 -> CWRK009-00-0184 -> depot
Distance of route: 0.0 km

Route for vehicle 1:
depot -> CWRK009-00-0044 -> WRK009-21-0000007 -> CWRK009-00-0064 -> WRK009-22-0000002 -> WRK009-22-0000002 -> CWRK009-00-0023 -> WRK009-21-0000103 -> CWRK009-00-0193 -> CWRK009-00-0033 -> WRK009-23-0000129 -> WRK009-19-0000018 -> CWRK009-00-0081 -> depot
Distance of route: 0.0 km

Route for vehicle 2:
depot -> WRK009-23-0000092 -> WRK009-23-0000145 -> WRK0

In [25]:
optimized_delivery_map

In [26]:
# saving interactive map in html
optimized_delivery_map.save('maps/Week 1.html')